In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('LinerRegression').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 13:18:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark

In [8]:
trainingData = spark.read.format('libsvm').load('../Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

22/12/03 10:25:02 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [ ]:
trainingData.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [ ]:
# Creating an instance of the ML model
# We can mention many more parameters 

lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction', maxIter=,  regParam=)

In [ ]:
lrModel = lr.fit(trainingData)

22/12/03 09:17:32 WARN Instrumentation: [09e1bce6] regParam is zero, which might cause numerical instability and overfitting.
22/12/03 09:17:33 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/12/03 09:17:33 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [ ]:
# Grabbing the coefficients
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [ ]:
# Grabbing the intercept

lrModel.intercept

0.14228558260358093

In [ ]:
# Stores summary of lots of important values

trainingSummary = lrModel.summary
trainingSummary.r2

0.027839179518600154

In [ ]:
# Splitting our data into Training and Test set

allData = spark.read.format('libsvm').load('../Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

# Splitting our data into 70 and 30 percent
splitObject = allData.randomSplit([0.7, 0.3])
print(splitObject)

trainData, testData = splitObject

22/12/03 09:28:38 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
[DataFrame[label: double, features: vector], DataFrame[label: double, features: vector]]


In [ ]:
# Training the Model on the trainData

correctModel = lr.fit(trainData)

correctModelSummary = correctModel.summary

22/12/03 09:31:40 WARN Instrumentation: [53f4f75f] regParam is zero, which might cause numerical instability and overfitting.


In [ ]:
# Now checking how well our Model performed using the testData
testResults = correctModel.evaluate(testData)

testResults.rootMeanSquaredError

In [ ]:
# We would actually want to train our model on unlabelled data

unlabelledData = allData.select(['features'])
unlabelledData.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [ ]:
# Transforming our unlabelled data to produce the predictions using the Features column
predictedData = correctModel.transform(unlabelledData)
predictedData.show(5)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 2.3909883686622213|
|(10,[0,1,2,3,4,5,...|-1.3301067276115592|
|(10,[0,1,2,3,4,5,...|0.17760720381421735|
|(10,[0,1,2,3,4,5,...| 1.1595313008116446|
|(10,[0,1,2,3,4,5,...|  3.042496126448815|
+--------------------+-------------------+
only showing top 5 rows

